In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
from sklearn.datasets import make_hastie_10_2
import matplotlib.pyplot as plt
import random
import math

""" HELPER FUNCTION: SIGMOID ========================================="""
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

""" HELPER FUNCTION: GET ERROR RATE ========================================="""
def get_error_rate(pred, Y):
    return sum(pred != Y) / float(len(Y))

""" HELPER FUNCTION: PRINT ERROR RATE ======================================="""
def print_error_rate(err):
    print('Error rate: Training: %.4f - Test: %.4f' % err)

""" HELPER FUNCTION: GENERIC CLASSIFIER ====================================="""
def generic_clf(Y_train, X_train, Y_test, X_test, clf):
    clf.fit(X_train,Y_train)
    pred_train = clf.predict(X_train)
    pred_test = clf.predict(X_test)
    return get_error_rate(pred_train, Y_train), \
           get_error_rate(pred_test, Y_test)
    
""" ADABOOST IMPLEMENTATION ================================================="""
def adaboost_clf(Y_train, X_train, Y_test, X_test, M, clf):
    n_train, n_test = len(X_train), len(X_test)
    # Initialize weights
    w = np.ones(n_train) / n_train
    #print(w)
    #print(np.shape(w))
    pred_train, pred_test = [np.zeros(n_train), np.zeros(n_test)]
    
    for i in range(M):
        # Fit a classifier with the specific weights
        clf.fit(X_train, Y_train, sample_weight = np.ones(n_train) / n_train)
        #np.reshape(w[0:125],(125))
        #print(X_train)
        pred_train_i = clf.predict(X_train)
        #print(np.shape(pred_train_i))
        pred_test_i = clf.predict(X_test)
        # Indicator function
        miss =np.zeros((125,1), dtype=float)
        x=(len(pred_train_i)-1)
        for i in range(x):
            #print(pred_train_i[x])    
            #print(Y_train[x])
            if(pred_train_i[x]!= Y_train[x]):
                miss[x]=1
        miss=np.asarray(miss)
        miss=np.reshape(miss, (125,1))
        #print(miss)
        # Equivalent with 1/-1 to update weights
        miss2 = [1 if x==1 else -1 for x in miss]
        # Error
        #print(np.shape(w))
        #print(np.shape(miss))        
        err_m = np.dot(w,miss) / sum(w)
        #print(err_m)# Alpha
        for i in range(len(err_m)-1):
            alpha_m[i] = 0.5 * np.log( (1 - err_m[i]))
            alpha_m[i] = alpha_m[i] / float(err_m[i])
            w[i] = np.multiply(w[i], np.exp([float(x) * alpha_m[i] for x in miss2]))
            pred_train[i] = [sum(x) for x in zip(pred_train[i], 
                                          [x * alpha_m[i] for x in pred_train_i])]
            pred_test[i] = [sum(x) for x in zip(pred_test[i], 
                                         [x * alpha_m[i] for x in pred_test_i])]
        #print(alpha_m)
        # New weights
        # Add to prediction
        
    
    pred_train, pred_test = np.sign(pred_train), np.sign(pred_test)
    # Return error rate in train and test set
    return get_error_rate(pred_train, Y_train), \
           get_error_rate(pred_test, Y_test)

""" PLOT FUNCTION ==========================================================="""
def plot_error_rate(er_train, er_test):
    df_error = pd.DataFrame([er_train, er_test]).T
    df_error.columns = ['Training', 'Test']
    plot1 = df_error.plot(linewidth = 3, figsize = (8,6),
            color = ['lightblue', 'darkblue'], grid = True)
    plot1.set_xlabel('Number of iterations', fontsize = 12)
    plot1.set_xticklabels(range(0,450,50))
    plot1.set_ylabel('Error rate', fontsize = 12)
    plot1.set_title('Error rate vs number of iterations', fontsize = 16)
    plt.axhline(y=er_test[0], linewidth=1, color = 'red', ls = 'dashed')

""" MAIN SCRIPT ============================================================="""
if __name__ == '__main__':
    
    # Read data
    #x, y = make_hastie_10_2()
    #df = pd.DataFrame(x)
    #df['Y'] = y

    x=np.zeros((8,5,5,2))
    print(np.shape(x))
    
    #y=np.ones((8,5,5))
    #y=np.reshape(2,2,2)
    #print(np.shape(y))    
  
    # Split into training and test set
    train, test = train_test_split(x, test_size = 0.3)
    X_train, Y_train = train[:,:,:,:], train[:,:,:,0]
    X_test, Y_test = test[:,:,:,:], test[:,:,:,0]
    
    print(np.shape(X_train))
    print(np.shape(Y_train))
    print(np.shape(X_test))
    print(np.shape(Y_test))
    
    X_train=np.reshape(X_train, ((125,2)))
    Y_train=np.reshape(Y_train, ((125,1)))
    X_test=np.reshape(X_test, ((75,2)))
    Y_test=np.reshape(Y_test, ((75,1)))

    Y_train=np.ones((125,1))
    Y_test=np.ones((75,1))
    
    print(np.shape(X_train))
    print(np.shape(Y_train))
    print(np.shape(X_test))
    print(np.shape(Y_test))

    #print(X_train)
    # Fit a simple decision tree first
    clf_tree = DecisionTreeClassifier(max_depth = 1, random_state = 1)
    er_tree = generic_clf(Y_train, X_train, Y_test, X_test, clf_tree)
    
    # Fit Adaboost classifier using a decision tree as base estimator
    # Test with different number of iterations
    er_train, er_test = [er_tree[0]], [er_tree[1]]
    x_range = range(10, 410, 10)
    for i in x_range:    
        er_i = adaboost_clf(Y_train, X_train, Y_test, X_test, i, clf_tree)
        er_train.append(er_i[0])
        er_test.append(er_i[1])
    
    # Compare error rate vs number of iterations
    plot_error_rate(er_train, er_test)